# Crypto Currencies - Time Series Forecasting
# <font color='blue'>0. Identification for Business Requirements</font> <a class="anchor" id="first-bullet"></a>

### A Short Description About the Dataset & It's Source


####  Dataset of Bitcoin Historical Data

## <font color= blue >1. Importing the Necessary Libraries</font> <a class="anchor" id="second-bullet"></a>

In [ ]:
# Implement Hyperparameter tunning on the available models.

# Start on creating the classes for the Hybrid Models
# Exponential Smoothing (ES)
# Multi-Layer Perceptron (MLP) 
# Recurrent Neural Network (RNN) 
# Temporal Fusion Transformer (TFT - Google)
# DeepAR
# N-BEATS


In [ ]:
#!pip install cryptocmd
#!pip install arch
#!pip install prophet
#!pip install torch
#!pip install tensorflow
#!pip install keras-tuner
#!pip install lightgbm
#!pip install xgboost
#!pip install prettytable
#!pip install bokeh
#!pip install pmdarima
#!pip uninstall statsmodels --y
#%pip install torch
#%pip install pytorch-forecasting

In [ ]:
# Standard library imports
import os
import time
import warnings
import sys
import requests
import pickle
import logging
import joblib


# Third party imports
import numpy as np
import pandas as pd
import scipy.stats as stats
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Data Preprocessing and Analysis
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller, grangercausalitytests
from statsmodels.tsa.seasonal import seasonal_decompose
from cryptocmd import CmcScraper

# Deep Learning and Machine Learning Libraries
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, TimeDistributed, Conv1D, MaxPooling1D, Flatten
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping
from keras.layers import ConvLSTM2D
from kerastuner.tuners import RandomSearch
from kerastuner import HyperModel
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import TimeSeriesSplit


# Metrics and Model Evaluation
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score, explained_variance_score
from prettytable import PrettyTable

# Plotting and Visualization
"""from bokeh.plotting import figure, show
from bokeh.models import HoverTool, ColumnDataSource, WheelZoomTool, Span, Range1d
from bokeh.layouts import column
from bokeh.models.widgets import Button
from bokeh.models.callbacks import CustomJS
from bokeh.themes import Theme"""

# Time Series Analysis and Forecasting
import pmdarima as pm
from arch import arch_model

# Miscellaneous
from joblib import dump, load
from math import pi

# Warnings and Logging
warnings.filterwarnings('ignore')
pd.set_option('display.float_format', '{:.3f}'.format)
logging.basicConfig(level=logging.INFO, format='%(asctime)s [%(levelname)s] - %(message)s')
logger = logging.getLogger(__name__)


# Block 1. - CryptoDate

In [ ]:
class CryptoData:
    """
    This class is responsible for fetching and formatting cryptocurrency data.

    Input:
        crypto_symbol (str): The symbol of the cryptocurrency, e.g., "BTC" for Bitcoin.

    Methods:
        get_cryptocmd_data: Fetches cryptocurrency data and returns a formatted DataFrame.
        get_display_data: Fetches and formats cryptocurrency data for display purposes.
    """
    def __init__(self, crypto_symbol: str):
        logging.info("Initializing CryptoData class")
        self.crypto_symbol = crypto_symbol

    def _fetch_cryptocmd_data(self) -> pd.DataFrame:
        logging.info(f"Fetching {self.crypto_symbol} data")
        try:
            scraper = CmcScraper(self.crypto_symbol)
            df = scraper.get_dataframe()
            if not {'Date', 'Open', 'High', 'Low', 'Close', 'Market Cap', 'Volume'}.issubset(df.columns):
                raise ValueError("Some expected columns are missing in the data.")
            logging.info(f"Data fetched successfully for {self.crypto_symbol}")
            return df
        except Exception as e:
            logging.error(f'An error occurred while fetching data: {e}')
            raise  # Propagate the exception

    def get_cryptocmd_data(self) -> pd.DataFrame:
        logging.info(f"Getting {self.crypto_symbol} data")
        df = self._fetch_cryptocmd_data()
        df.set_index('Date', inplace=True)
        df.index = pd.to_datetime(df.index)
        logging.info(f"Data obtained successfully for {self.crypto_symbol}")
        return df.round(2)

    @staticmethod
    def _format_monetary_value(value: float) -> str:
        return "${:,.2f}".format(value)

    @staticmethod
    def _format_volume_value(value: float) -> str:
        if value > 1e9:
            return "{:.2f}B".format(value/1e9)
        elif value > 1e6:
            return "{:.2f}M".format(value/1e6)
        else:
            return "{:,.2f}".format(value)

    def get_display_data(self) -> pd.DataFrame:
        logging.info(f"Formatting display data for {self.crypto_symbol}")
        display_df = self.get_cryptocmd_data().copy()
        monetary_columns = ['Open', 'High', 'Low', 'Close']
        display_df[monetary_columns] = display_df[monetary_columns].applymap(self._format_monetary_value)

        # Check and apply volume formatting only if they are of numeric type
        volume_like_columns = ['Volume', 'Market Cap']
        for column in volume_like_columns:
            if pd.api.types.is_numeric_dtype(display_df[column]):
                display_df[column] = display_df[column].apply(self._format_volume_value)

        logging.info(f"Display data formatted successfully for {self.crypto_symbol}")
        return display_df


In [ ]:
crypto = CryptoData("BTC")
data = crypto.get_cryptocmd_data()
data

# ------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# Block 2 - Crypto analytics
class CryptoAnalytics:
    """
    This class is responsible for performing analytics on cryptocurrency data.

    Input:
        df (pd.DataFrame): The DataFrame containing the cryptocurrency data.

    Methods:
        get_all_time_records: Returns the all-time high and low closing prices.
        get_yearly_analysis: Returns yearly high, low, average, and variation.
        get_monthly_analysis: Returns monthly high, low, average, and variation.
        get_weekly_analysis: Returns weekly high, low, average, and variation.
    """
    def __init__(self, crypto_data: CryptoData):
        logger.info("Initializing CryptoAnalytics class")
        self.df = crypto_data.get_cryptocmd_data()

    def format_and_round(self, number):
        return f'{number:,.2f}'

    def apply_custom_formatting(self, data):
        logger.info("Applying custom formatting")
        ordered_columns = [
            'Open_first', 'Close_last', 'variation_$_abs',
            'variation_%_rel', 'Close_min', 'Close_mean', 'Close_max'
        ]
        data = data[ordered_columns]
        return data

    def get_all_time_records(self):
        logger.info("Getting all time records")
        all_time_high = self.df['Close'].max()
        all_time_low = self.df['Close'].min()
        all_time_high_date = self.df['Close'].idxmax().strftime('%Y-%m-%d')
        all_time_low_date = self.df['Close'].idxmin().strftime('%Y-%m-%d')
        return all_time_high, all_time_low, all_time_high_date, all_time_low_date

    def get_yearly_analysis(self):
        logger.info("Getting yearly analysis")
        yearly_data = self.df.resample('Y').agg({'Close': ['last', 'mean', 'max', 'min'], 'Open': 'first'})
        yearly_data.columns = yearly_data.columns.map('_'.join).str.strip('_')
        yearly_data = self.calculate_variation(yearly_data)
        return self.apply_custom_formatting(yearly_data)

    def get_monthly_analysis(self):
        logger.info("Getting monthly analysis")
        monthly_data = self.df.resample('M').agg({'Close': ['last', 'mean', 'max', 'min'], 'Open': 'first'})
        monthly_data.columns = monthly_data.columns.map('_'.join).str.strip('_')
        monthly_data = self.calculate_variation(monthly_data)
        return self.apply_custom_formatting(monthly_data)

    def get_weekly_analysis(self):
        logger.info("Getting weekly analysis")
        weekly_data = self.df.resample('W').agg({'Close': ['last', 'mean', 'max', 'min'], 'Open': 'first'})
        weekly_data.columns = weekly_data.columns.map('_'.join).str.strip('_')
        weekly_data = self.calculate_variation(weekly_data)
        return self.apply_custom_formatting(weekly_data)

    def calculate_variation(self, data):
        logger.info("Calculating variation")
        data['variation_$_abs'] = data['Close_last'] - data['Open_first']
        data['variation_%_rel'] = ((data['Close_last'] - data['Open_first']) / data['Open_first']) * 100
        return data

    def display_all_analyses(self):
        logger.info("Displaying all analyses")
        all_time_high, all_time_low, all_time_high_date, all_time_low_date = self.get_all_time_records()
        print(f"All Time High: {self.format_and_round(all_time_high)} (Date: {all_time_high_date})")
        print(f"All Time Low: {self.format_and_round(all_time_low)} (Date: {all_time_low_date})")

        print("\nYearly Analysis:")
        display(self.get_yearly_analysis())

        print("\nMonthly Analysis:")
        display(self.get_monthly_analysis())

        print("\nWeekly Analysis:")
        display(self.get_weekly_analysis())


In [ ]:
# Create an instance of CryptoAnalytics
analytics = CryptoAnalytics(crypto)
# Display all analyses
analytics.display_all_analyses()

# ------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
crypto = CryptoData("BTC")
data = crypto.get_cryptocmd_data()

# ------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
class TimeSeriesAnalysis:
    """
    This class provides methods for performing time series analysis on cryptocurrency data.

    Input:
        data (pd.Series or array-like): Time series data to be analyzed.

    Methods:
        check_stationarity: Checks the stationarity of the time series using the Augmented Dickey-Fuller test.
        check_autocorrelation: Plots the autocorrelation and partial autocorrelation functions of the time series.
        check_volatility: Fits a GARCH(1,1) model to the time series and prints the model summary.
    """
    def __init__(self, data, target):
        logger.info("Initializing TimeSeriesAnalysis class")
        self.data = data
        self.target = target
        
    def check_stationarity(self):
        logger.info("Checking stationarity of the time series")
        result = adfuller(self.data[self.target])
        print('ADF Statistic: %f' % result[0])
        print('p-value: %f' % result[1])
        print('Critical Values:')
        for key, value in result[4].items():
            print('\t%s: %.3f' % (key, value))
        if result[1] <= 0.05:
            print('The series is likely stationary.')
        else:
            print('The series is likely non-stationary.')

    def check_autocorrelation(self):
        logger.info("Checking autocorrelation of the time series")
        fig, (ax1, ax2) = plt.subplots(1,2, figsize=(20,6))
        data1D = self.data[self.target].values   # Convert DataFrame to 1D numpy array
        plot_acf(data1D, lags=50, alpha=0.05, ax=ax1)
        plt.title("ACF for Close Price", size=20)

        plot_pacf(data1D, lags=50, alpha=0.05, method='ols', ax=ax2)
        plt.title("PACF for Close Price -- Daily", size=20)


    def check_volatility(self):
        logger.info("Checking volatility of the time series")
        model = arch_model(self.data[self.target], vol='Garch', p=1, q=1)
        model_fit = model.fit()
        print(model_fit.summary())
    
    def decompose_time_series(self, model='additive', period=30):
        logger.info("Decomposing the time series")
        result = seasonal_decompose(self.data[self.target], model=model, period=period)
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12, 8))
        result.observed.plot(ax=ax1)
        ax1.set_title('Observed')
        result.trend.plot(ax=ax2)
        ax2.set_title('Trend')
        result.seasonal.plot(ax=ax3)
        ax3.set_title('Seasonal')
        result.resid.plot(ax=ax4)
        ax4.set_title('Residual')
        
        plt.tight_layout()
        plt.show()
        
    def test_granger_causality(self, other_column, maxlag=30):
        logger.info("Testing Granger causality")
        other_data = self.data[other_column].values
        target_data = self.data[self.target].values
        data = np.column_stack((target_data, other_data))
        result = grangercausalitytests(data, maxlag=maxlag)
        return result
 

In [ ]:
# Create an instance of the TimeSeriesAnalysis class
tsa = TimeSeriesAnalysis(data,target='Close')
tsa.check_stationarity()
tsa.check_volatility()
tsa.check_autocorrelation()
tsa.decompose_time_series()
tsa.test_granger_causality('Open', maxlag=30)

# ------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
class DataPreprocessing:
    def __init__(self, data, target_columns, seq_length=5, scaler_path='./scaler.pkl'):
        logger.info("Initializing DataPreprocessing class")
        
        # Check if target_columns exist in data
        if not set(target_columns).issubset(data.columns):
            logger.error(f"Target columns {target_columns} not found in data.")
            raise ValueError(f"Target columns {target_columns} not found in data.")
        
        self.data = data.sort_index(ascending=True)
        self.target_columns = target_columns
        self.seq_length = seq_length
        self.scaler = StandardScaler()
        self.scaler_path = scaler_path

        # If a scaler file exists, load it. Otherwise, fit a new scaler
        if os.path.exists(self.scaler_path):
            self.load_scaler()
            self.data_scaled = self.scaler.transform(self.data[self.target_columns])
        else:
            self.data_scaled = self.scaler.fit_transform(self.data[self.target_columns])
            self.save_scaler()

    def save_scaler(self):
        logger.info("Saving scaler")
        joblib.dump(self.scaler, self.scaler_path)
        logger.info(f"Scaler saved to {self.scaler_path}")

    def load_scaler(self):
        logger.info("Loading scaler")
        try:
            self.scaler = joblib.load(self.scaler_path)
            logger.info(f"Scaler loaded from {self.scaler_path}")
        except Exception as e:
            logger.error(f"Error loading scaler from {self.scaler_path}: {e}")
            raise

    def split_data(self, test_size=0.2):
        logger.info("Splitting data")
        train_data, test_data = train_test_split(self.data_scaled, test_size=test_size, shuffle=False)
        return train_data, test_data

    def prepare_data_rnn(self, data):
        logger.info("Preparing data for RNN")
        X, y = [], []
        for i in range(self.seq_length, len(data)):
            X.append(data[i-self.seq_length:i])
            y.append(data[i, 0])
        X, y = np.array(X), np.array(y)
        logger.info(f"RNN Data Shape - X: {X.shape}, y: {y.shape}")
        return X, y   

    def prepare_data_for_tree_based_model(self, target_column='Close', test_size=0.2, scale=False):
        logger.info("Preparing data for tree-based model")
        X = self.data.drop(target_column, axis=1)
        y = self.data[target_column]

        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, shuffle=False)

        if scale:
            scaler = StandardScaler()
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)
            
        logger.info(f"Tree Data Shape - X: {X_train.shape}, {y_train.shape} , y: {X_test.shape}, {y_test.shape}")
        return X_train, y_train, X_test, y_test

    def visualize_split(self, train_data, test_data):
        logger.info("Visualizing data split")
        plt.figure(figsize=(14, 7))
        plt.plot(np.arange(len(train_data)), self.scaler.inverse_transform(train_data)[:, 0], color='blue', label='Training Data')
        plt.plot(np.arange(len(train_data), len(train_data) + len(test_data)), self.scaler.inverse_transform(test_data)[:, 0], color='orange', label='Testing Data')
        plt.xlabel('Time Step')
        plt.ylabel('Normalized Value')
        plt.legend()
        plt.show()

    def prepare_data_SARIMAX(self, target_column='Close', exog_columns=None, test_size=0.2):
        logger.info("Preparing data for SARIMAX")
        y = self.data[target_column]
        X = self.data[exog_columns] if exog_columns else None

        y_train, y_test = train_test_split(y, test_size=test_size, shuffle=False)
        X_train, X_test = train_test_split(X, test_size=test_size, shuffle=False) if X is not None else (None, None)

        logger.info(f"SARIMAX Data Shape - y: {y_train.shape}, {y_test.shape}")
        return y_train, y_test, X_train, X_test

    def prepare_data_deepar(self):
        logger.info("Preparing data for DeepAR")
        # Add your preferred method to prepare data for AutoTS model
        pass

    def prepare_data_deeptime(self):
        logger.info("Preparing data for DeepTime")
        # Add your preferred method to prepare data for DeepTime model
        pass

In [ ]:
# Assuming 'Close' is your target column
preprocessor = DataPreprocessing(data, ['Close'])
scaled_data = preprocessor.data_scaled
train_data, test_data = preprocessor.split_data(test_size=0.2)
#preprocessor.visualize_split(train_data, test_data)
X_train_rnn, y_train_rnn = preprocessor.prepare_data_rnn(train_data)
X_test_rnn, y_test_rnn = preprocessor.prepare_data_rnn(test_data)

In [ ]:
preprocessor = DataPreprocessing(data, ['Close'])
scaled_data = preprocessor.data_scaled
train_data, test_data = preprocessor.split_data(test_size=0.2)
# For Linear Regression
X_train_lr, y_train_lr, X_test_lr, y_test_lr = preprocessor.prepare_data_for_tree_based_model(target_column='Close', test_size=0.2, scale=True)
# For XGBoost
X_train_xgb, y_train_xgb, X_test_xgb, y_test_xgb = preprocessor.prepare_data_for_tree_based_model(target_column='Close', test_size=0.2, scale=False)
# For LightGBM
X_train_lgb, y_train_lgb, X_test_lgb, y_test_lgb = preprocessor.prepare_data_for_tree_based_model(target_column='Close', test_size=0.2, scale=False)

# ------------------------------------------------------------------------------------------------------------------------------------------

# Block X - LSTM Class

In [ ]:
class LSTM_Models:
    def __init__(self, input_shape, units, dropout, dense_units, optimizer, scaler, X_train, y_train, X_test, y_test,cross_val=False):
        logging.info("Initializing LSTM model")
        self.model = Sequential()
        for i, unit in enumerate(units):
            return_sequences = True if i < len(units) - 1 else False
            self.model.add(LSTM(units=unit, input_shape=input_shape, return_sequences=return_sequences))
            self.model.add(Dropout(dropout))
        self.model.add(Dense(units=dense_units))
        self.model.compile(optimizer=optimizer, loss='mean_squared_error')
        self.model.summary()
        self.cross_val = cross_val
        self.scaler = scaler
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
        
        self.params = {
            'input_shape': input_shape,
            'units': units,
            'dropout': dropout,
            'dense_units': dense_units,
            'optimizer': optimizer
        }

    def train_model(self, epochs=100, batch_size=50, early_stopping=True):
        logging.info("Training LSTM model")
        callbacks = [EarlyStopping(monitor='val_loss', patience=10)] if early_stopping else None
        if self.cross_val:
            tscv = TimeSeriesSplit(n_splits=5)
            self.history = []
            fold_no = 1
            for train, val in tscv.split(self.X_train):
                logging.info(f"Training on fold {fold_no}")
                history = self.model.fit(self.X_train[train], self.y_train[train], epochs=epochs, 
                                         batch_size=batch_size, validation_data=(self.X_train[val], self.y_train[val]), 
                                         callbacks=callbacks, shuffle=False)
                self.history.append(history)
                logging.info(f"Done with fold {fold_no}")
                fold_no += 1
        else:
            self.history = self.model.fit(self.X_train, self.y_train, epochs=epochs, 
                                          batch_size=batch_size, validation_split=0.2, 
                                          callbacks=callbacks, shuffle=False)
        logging.info("Training completed")

    def make_predictions(self):
        logging.info("Making predictions with LSTM model")
        self.train_predictions = self.model.predict(self.X_train)
        self.test_predictions = self.model.predict(self.X_test)

        # Unscaled the predictions
        self.train_predictions = self.scaler.inverse_transform(self.train_predictions)
        self.test_predictions = self.scaler.inverse_transform(self.test_predictions)

        self.train_comparison_df = pd.DataFrame({'Actual': self.scaler.inverse_transform(self.y_train.reshape(-1, 1)).flatten(), 
                                                 'Predicted': self.train_predictions.squeeze()})
        self.test_comparison_df = pd.DataFrame({'Actual': self.scaler.inverse_transform(self.y_test.reshape(-1, 1)).flatten(), 
                                                'Predicted': self.test_predictions.squeeze()})
        logging.info("Predictions made")

    def evaluate_model(self):
        logging.info("Evaluating LSTM model")
        metrics = {'RMSE': mean_squared_error, 'R2 Score': r2_score, 
                   'MAE': mean_absolute_error, 'Explained Variance': explained_variance_score}

        evaluation = {}
        for name, metric in metrics.items():
            if name == 'RMSE':
                train_evaluation = metric(self.train_comparison_df['Actual'], 
                                          self.train_comparison_df['Predicted'], 
                                          squared=False)
                test_evaluation = metric(self.test_comparison_df['Actual'], 
                                         self.test_comparison_df['Predicted'], 
                                         squared=False)
            else:
                train_evaluation = metric(self.train_comparison_df['Actual'], 
                                          self.train_comparison_df['Predicted'])
                test_evaluation = metric(self.test_comparison_df['Actual'], 
                                         self.test_comparison_df['Predicted'])
            evaluation[name] = {'Train': train_evaluation, 'Test': test_evaluation}

        self.evaluation_df = pd.DataFrame(evaluation)
        logging.info("Evaluation completed")
        return self.evaluation_df

    def plot_history(self):
        logging.info("Plotting training history")
        plt.figure(figsize=(14, 7))

        if self.cross_val:
            # Loop through the history of each fold
            for i, history in enumerate(self.history):
                plt.plot(history.history['loss'], label=f'Training Loss Fold {i+1}')
                plt.plot(history.history['val_loss'], label=f'Validation Loss Fold {i+1}')
        else:
            plt.plot(self.history.history['loss'], label='Training Loss')
            plt.plot(self.history.history['val_loss'], label='Validation Loss')

        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
        plt.show()
        logging.info("Plotting completed")

    def plot_predictions(self):
        logging.info("Plotting predictions")
        fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(14, 7))

        axes[0].plot(self.train_comparison_df['Actual'], label='Actual')
        axes[0].plot(self.train_comparison_df['Predicted'], label='Predicted')
        axes[0].set_title('Training Data')
        axes[0].legend()

        axes[1].plot(self.test_comparison_df['Actual'], label='Actual')
        axes[1].plot(self.test_comparison_df['Predicted'], label='Predicted')
        axes[1].set_title('Testing Data')
        axes[1].legend()

        plt.show()
        logging.info("Plotting completed")
            
    def save_model(self):
        # Create a model name based on the parameters
        model_name = f"LSTM_model_{str(self.params).replace(' ', '').replace(':', '').replace(',', '_')}.h5"
        model_path = os.path.join("trained_models", model_name)

        # Save the model
        self.model.save(model_path)
        logging.info(f"Model saved at {model_path}")

    @staticmethod
    def load_model(model_path):
        # Load the model
        loaded_model = load_model(model_path)
        logging.info(f"Model loaded from {model_path}")
        return loaded_model

# Block X - LSTM Models 

In [ ]:
# After preparing your data with the DataPreprocessing class...
lstm_model = LSTM_Models(input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2]), 
                         units=[256, 128], 
                         dropout=0.2, 
                         dense_units=1, 
                         optimizer='adam', 
                         scaler=preprocessor.scaler, 
                         X_train=X_train_rnn, 
                         y_train=y_train_rnn, 
                         X_test=X_test_rnn, 
                         y_test=y_test_rnn,
                        cross_val=False)

lstm_model.train_model(epochs=100, batch_size=32, early_stopping=True)
lstm_model.save_model()  # save the model
lstm_model.make_predictions()

In [ ]:
lstm_model.evaluate_model()
lstm_model.plot_history()
lstm_model.plot_predictions()

In [ ]:
# After preparing your data with the DataPreprocessing class...
lstm_model1 = LSTM_Models(input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2]), 
                         units=[256], 
                         dropout=0.2, 
                         dense_units=1, 
                         optimizer='adam', 
                         scaler=preprocessor.scaler, 
                         X_train=X_train_rnn, 
                         y_train=y_train_rnn, 
                         X_test=X_test_rnn, 
                         y_test=y_test_rnn)

lstm_model1.train_model(epochs=100, batch_size=32, early_stopping=True)
lstm_model.save_model()  # save the model
lstm_model1.make_predictions()

In [ ]:
lstm_model1.evaluate_model()
lstm_model1.plot_history()
lstm_model1.plot_predictions()

# ------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
Stop

# Block X - Hardcoded Grid-Search For LSTM

In [ ]:
best_score = float('inf')
best_model = None
best_params = None

# Define the grid
layer_grid = [[30], [50], [30, 30], [50, 50], [30, 50, 30]]

for layers in layer_grid:
    print(f"Training model with layers: {layers}")
    
    # Train the model
    model = LSTM_Models(input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2]), 
                        units=layers, 
                        dropout=0.2, 
                        dense_units=1, 
                        optimizer='adam', 
                        scaler=preprocessor.scaler, 
                         X_train=X_train_rnn, 
                         y_train=y_train_rnn, 
                         X_test=X_test_rnn, 
                         y_test=y_test_rnn)
    model.train_model(epochs=100, batch_size=32, early_stopping=True)
    model.make_predictions()
    model.evaluate_model()

    # Check if this model is the best so far
    current_score = model.evaluation_df.loc['Test', 'RMSE']
    if current_score < best_score:
        best_score = current_score
        best_model = model
        best_params = layers

print(f"Best model has layers: {best_params} with RMSE: {best_score}")


# ------------------------------------------------------------------------------------------------------------------------------------------

# Block X - Keras Tuner Grid-Search For LSTM

In [ ]:
class LSTMHyperModel(HyperModel):
    def __init__(self, input_shape, scaler):
        self.input_shape = input_shape
        self.scaler = scaler

    def build(self, hp):
        model = Sequential()
        for i in range(hp.Int('num_layers', 1, 3)):
            model.add(LSTM(units=hp.Int('units_' + str(i), min_value=30, max_value=80, step=10),
                           activation=hp.Choice('activation_'+str(i), ['relu', 'tanh']),
                           input_shape=self.input_shape,
                           return_sequences=True if i < hp.Int('num_layers', 1, 3) - 1 else False))
            model.add(Dropout(rate=hp.Float('dropout_'+str(i), min_value=0.0, max_value=0.5, step=0.1)))
        model.add(Dense(units=hp.Int('dense_units', 1, 3), activation=hp.Choice('dense_activation', ['relu', 'linear'])))
        
        model.compile(optimizer=hp.Choice('optimizer', ['adam', 'sgd']),
                      loss='mean_squared_error')
        
        return model


In [ ]:
hypermodel = LSTMHyperModel(input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2]), scaler=preprocessor.scaler)

tuner = RandomSearch(
    hypermodel,
    objective='val_loss',
    max_trials=10,  # adjust based on resources
    executions_per_trial=2,  # adjust based on resources
    directory='random_search',
    project_name='lstm'
)

tuner.search(X_train_rnn, y_train_rnn, epochs=50, validation_split=0.2)

In [ ]:
best_model = tuner.get_best_models()[0]
best_hyperparameters = tuner.get_best_hyperparameters()[0]

In [ ]:
# Display the best model's architecture
best_model.summary()
# Display the best hyperparameters
print(best_hyperparameters.values)

# Block 8 - Linear Model

# ------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
class Linear_Regression:
    def __init__(self, X_train, y_train, X_test, y_test,cross_val=False):
        logger.info("Initializing Linear_Regression class")
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
        self.cross_val = cross_val
        self.model = LinearRegression()

    def train_model(self):
        logger.info("Training Linear Regression model")
        
        if self.cross_val:
            tscv = TimeSeriesSplit(n_splits=5)
            fold_no = 1
            for train_index, test_index in tscv.split(self.X_train):
                logger.info(f"Performing cross validation on fold {fold_no}")
                cv_train_x, cv_val_x = self.X_train[train_index], self.X_train[test_index]
                cv_train_y, cv_val_y = self.y_train[train_index], self.y_train[test_index]
                self.model.fit(cv_train_x, cv_train_y)
                cv_val_predictions = self.model.predict(cv_val_x)
                cv_val_score = mean_squared_error(cv_val_y, cv_val_predictions, squared=False)
                logger.info(f"Cross-validation score on fold {fold_no}: {cv_val_score}")
                fold_no += 1
        else:
            self.model.fit(self.X_train, self.y_train)

    def make_predictions(self):
        logger.info("Making predictions with Linear Regression model")
        self.train_predictions = self.model.predict(self.X_train)
        self.test_predictions = self.model.predict(self.X_test)

    def evaluate_model(self):
        logger.info("Evaluating Linear Regression model")
        metrics = {'RMSE': mean_squared_error, 'R2 Score': r2_score, 
                   'MAE': mean_absolute_error, 'Explained Variance': explained_variance_score}

        evaluation = {}
        for name, metric in metrics.items():
            if name == 'RMSE':
                train_evaluation = metric(self.y_train, self.train_predictions, squared=False)
                test_evaluation = metric(self.y_test, self.test_predictions, squared=False)
            else:
                train_evaluation = metric(self.y_train, self.train_predictions)
                test_evaluation = metric(self.y_test, self.test_predictions)
            evaluation[name] = {'Train': train_evaluation, 'Test': test_evaluation}

        self.evaluation_df = pd.DataFrame(evaluation)
        return self.evaluation_df

    def plot_predictions(self):
        logger.info("Plotting predictions")
        fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(14, 7))

        ax[0].plot(self.y_train, label='Actual')
        ax[0].plot(self.y_train.index, self.train_predictions, label='Predicted')
        ax[0].set_title('Train Data: Actual vs Predicted')
        ax[0].legend()

        ax[1].plot(self.y_test, label='Actual')
        ax[1].plot(self.y_test.index, self.test_predictions, label='Predicted')
        ax[1].set_title('Test Data: Actual vs Predicted')
        ax[1].legend()

        plt.tight_layout()
        plt.show()
        
    def save_model(self):
        model_name = "linear_regression.joblib"
        model_path = os.path.join("trained_models", model_name)

        # Save the model
        dump(self.model, model_path)
        logger.info(f"Model saved at {model_path}")

    @staticmethod
    def load_model(model_path):
        # Load the model
        loaded_model = load(model_path)
        logger.info(f"Model loaded from {model_path}")
        return loaded_model



In [ ]:
lr_model = Linear_Regression(X_train_lr, y_train_lr, X_test_lr, y_test_lr, cross_val=True)
lr_model.train_model()
lr_model.save_model()  # save the model
lr_model.make_predictions()
evaluation_df = lr_model.evaluate_model()
print(evaluation_df)
lr_model.plot_predictions()


# ------------------------------------------------------------------------------------------------------------------------------------------

# Block 8 - XGBoost Model

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

class XGBoostModel:
    def __init__(self, X_train, y_train, X_test, y_test, cross_val=False):
        logger.info("Initializing XGBoostModel class")
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
        self.cross_val = cross_val
        self.model = XGBRegressor(objective='reg:squarederror')

    def train_model(self):
        logger.info("Training XGBoost model")

        if self.cross_val:
            tscv = TimeSeriesSplit(n_splits=5)
            fold_no = 1
            for train_index, test_index in tscv.split(self.X_train):
                logger.info(f"Performing cross validation on fold {fold_no}")
                cv_train_x, cv_val_x = self.X_train.iloc[train_index], self.X_train.iloc[test_index]
                cv_train_y, cv_val_y = self.y_train.iloc[train_index], self.y_train.iloc[test_index]
                self.model.fit(cv_train_x, cv_train_y)
                cv_val_predictions = self.model.predict(cv_val_x)
                cv_val_score = mean_squared_error(cv_val_y, cv_val_predictions, squared=False)
                logger.info(f"Cross-validation score on fold {fold_no}: {cv_val_score}")
                fold_no += 1
        else:
            self.model.fit(self.X_train, self.y_train)

    def make_predictions(self):
        self.train_predictions = self.model.predict(self.X_train)
        self.test_predictions = self.model.predict(self.X_test)

    def evaluate_model(self):
        metrics = {'RMSE': mean_squared_error, 'R2 Score': r2_score, 
                   'MAE': mean_absolute_error, 'Explained Variance': explained_variance_score}

        evaluation = {}
        for name, metric in metrics.items():
            if name == 'RMSE':
                train_evaluation = metric(self.y_train, self.train_predictions, squared=False)
                test_evaluation = metric(self.y_test, self.test_predictions, squared=False)
            else:
                train_evaluation = metric(self.y_train, self.train_predictions)
                test_evaluation = metric(self.y_test, self.test_predictions)
            evaluation[name] = {'Train': train_evaluation, 'Test': test_evaluation}

        self.evaluation_df = pd.DataFrame(evaluation)
        return self.evaluation_df

    def plot_predictions(self):
        fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(14, 7))

        ax[0].plot(self.y_train, label='Actual')
        ax[0].plot(self.y_train.index, self.train_predictions, label='Predicted')
        ax[0].set_title('Train Data: Actual vs Predicted')
        ax[0].legend()

        ax[1].plot(self.y_test, label='Actual')
        ax[1].plot(self.y_test.index, self.test_predictions, label='Predicted')
        ax[1].set_title('Test Data: Actual vs Predicted')
        ax[1].legend()

        plt.tight_layout()
        plt.show()

    def save_model(self):
        model_name = "xgboost_model.joblib"
        model_path = os.path.join("trained_models", model_name)

        # Save the model
        dump(self.model, model_path)
        logger.info(f"Model saved at {model_path}")

    @staticmethod
    def load_model(model_path):
        # Load the model
        loaded_model = load(model_path)
        logger.info(f"Model loaded from {model_path}")
        return loaded_model

In [ ]:
xgb_model = XGBoostModel(X_train_xgb, y_train_xgb, X_test_xgb, y_test_xgb,cross_val=False)
xgb_model.train_model()
xgb_model.save_model()  # save the model
xgb_model.make_predictions()
evaluation_df = xgb_model.evaluate_model()
print(evaluation_df)
xgb_model.plot_predictions()

# ------------------------------------------------------------------------------------------------------------------------------------------

# Block 8 - LightGBM Model

In [ ]:
class LightGBMModel:
    def __init__(self, X_train, y_train, X_test, y_test, cross_val=False):
        logger.info("Initializing LightGBMModel class")
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
        self.cross_val = cross_val
        self.model = LGBMRegressor()

    def train_model(self):
        logger.info("Training LightGBM model")

        if self.cross_val:
            tscv = TimeSeriesSplit(n_splits=5)
            fold_no = 1
            for train_index, test_index in tscv.split(self.X_train):
                logger.info(f"Performing cross validation on fold {fold_no}")
                cv_train_x, cv_val_x = self.X_train.iloc[train_index], self.X_train.iloc[test_index]
                cv_train_y, cv_val_y = self.y_train.iloc[train_index], self.y_train.iloc[test_index]
                self.model.fit(cv_train_x, cv_train_y)
                cv_val_predictions = self.model.predict(cv_val_x)
                cv_val_score = mean_squared_error(cv_val_y, cv_val_predictions, squared=False)
                logger.info(f"Cross-validation score on fold {fold_no}: {cv_val_score}")
                fold_no += 1
        else:
            self.model.fit(self.X_train, self.y_train)

    def make_predictions(self):
        logger.info("Making predictions with LightGBM model")
        self.train_predictions = self.model.predict(self.X_train)
        self.test_predictions = self.model.predict(self.X_test)

    def evaluate_model(self):
        logger.info("Evaluating LightGBM model")
        metrics = {'RMSE': mean_squared_error, 'R2 Score': r2_score, 
                   'MAE': mean_absolute_error, 'Explained Variance': explained_variance_score}

        evaluation = {}
        for name, metric in metrics.items():
            if name == 'RMSE':
                train_evaluation = metric(self.y_train, self.train_predictions, squared=False)
                test_evaluation = metric(self.y_test, self.test_predictions, squared=False)
            else:
                train_evaluation = metric(self.y_train, self.train_predictions)
                test_evaluation = metric(self.y_test, self.test_predictions)
            evaluation[name] = {'Train': train_evaluation, 'Test': test_evaluation}

        self.evaluation_df = pd.DataFrame(evaluation)
        return self.evaluation_df

    def plot_predictions(self):
        logger.info("Plotting predictions")
        fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(14, 7))

        ax[0].plot(self.y_train, label='Actual')
        ax[0].plot(self.y_train.index, self.train_predictions, label='Predicted')
        ax[0].set_title('Train Data: Actual vs Predicted')
        ax[0].legend()

        ax[1].plot(self.y_test, label='Actual')
        ax[1].plot(self.y_test.index, self.test_predictions, label='Predicted')
        ax[1].set_title('Test Data: Actual vs Predicted')
        ax[1].legend()

        plt.tight_layout()
        plt.show()

    def save_model(self):
        model_name = "lightgbm_model.joblib"
        model_path = os.path.join("trained_models", model_name)

        # Save the model
        dump(self.model, model_path)
        logger.info(f"Model saved at {model_path}")

    @staticmethod
    def load_model(model_path):
        # Load the model
        loaded_model = load(model_path)
        logger.info(f"Model loaded from {model_path}")
        return loaded_model


In [ ]:
lgb_model = LightGBMModel(X_train_lgb, y_train_lgb, X_test_lgb, y_test_lgb, cross_val=False)
lgb_model.train_model()
lgb_model.save_model()  # save the model
lgb_model.make_predictions()
evaluation_df = lgb_model.evaluate_model()
print(evaluation_df)
lgb_model.plot_predictions()


# ------------------------------------------------------------------------------------------------------------------------------------------

# Block 8 - AutoArima

In [ ]:
# Assuming 'Close' is your target column
preprocessor = DataPreprocessing(data, ['Close'])
scaled_data = preprocessor.data_scaled
train_data, test_data = preprocessor.split_data(test_size=0.2)
y_train, y_test, X_train, X_test = preprocessor.prepare_data_SARIMAX('Close', ['Volume'])
y_train, y_test, X_train, X_test = preprocessor.prepare_data_SARIMAX('Close')

In [ ]:
class SARIMAModel:
    def __init__(self, train, test, exog_train=None, exog_test=None):
        logger.info("Initializing SARIMAModel class")
        self.train = train
        self.test = test
        self.exog_train = exog_train
        self.exog_test = exog_test
        self.model = None
        self.predictions = None

    def auto_fit(self, seasonal=True, m=12, trace=True, error_action='ignore', suppress_warnings=True):
        logger.info("Starting auto_fit...")
        try:
            self.model = pm.auto_arima(self.train, exogenous=self.exog_train, seasonal=seasonal, m=m, trace=trace, error_action=error_action, suppress_warnings=suppress_warnings)
            logger.info("Auto_fit completed successfully.")
        except Exception as e:
            logger.error(f"Auto_fit failed with error: {e}")
            raise

    def make_predictions(self,start, end):
        if self.model is None:
            logger.error("Model not fitted yet.")
            return
        logger.info("Making predictions...")
        self.predictions = self.model.predict(n_periods=len(self.test), exogenous=self.exog_test)
        return self.predictions
    
    def evaluate_model(self):
        if self.model is None or self.predictions is None:
            logger.error("Model not fitted or predictions not made yet.")
            return
        mse = mean_squared_error(self.test, self.predictions)
        mae = mean_absolute_error(self.test, self.predictions)
        r2 = r2_score(self.test, self.predictions)
        
        logger.info(f'MSE: {mse}')
        logger.info(f'MAE: {mae}')
        logger.info(f'R2 Score: {r2}')

    def plot_predictions(self):
        if self.model is None or self.predictions is None:
            logger.error("Model not fitted or predictions not made yet.")
            return
        logger.info("Plotting predictions")
        plt.figure(figsize=(10,5))
        plt.plot(self.test, label='Actual')
        plt.plot(self.predictions, label='Predicted')
        plt.title('Test Data vs Predicted Data')
        plt.legend()
        plt.show()

    def save_model(self):
        model_name = "sarima_model.joblib"
        model_path = os.path.join("trained_models", model_name)

        # Save the model
        dump(self.model, model_path)
        logger.info(f"Model saved at {model_path}")

    @staticmethod
    def load_model(model_path):
        # Load the model
        loaded_model = load(model_path)
        logger.info(f"Model loaded from {model_path}")
        return loaded_model

In [ ]:
# Initialize the SARIMAModel class
model = SARIMAModel(y_train, y_test)# Fit the SARIMA model with seasonal set to True and m set to 12 for monthly data
model.auto_fit(seasonal=True, m=12)
model.save_model()  # save the model
# Make predictions for the length of the test set
model.make_predictions(start=len(y_train), end=len(y_train) + len(y_test) - 1)
# Evaluate the model
model.evaluate_model()
# Plot the actual vs predicted values
model.plot_predictions()

In [ ]:
class DataPreprocessing:
    def __init__(self, data, target_columns, model_name, seq_length=5):
        logger.info("Initializing DataPreprocessing class")
        
        # Check if target_columns exist in data
        if not set(target_columns).issubset(data.columns):
            logger.error(f"Target columns {target_columns} not found in data.")
            raise ValueError(f"Target columns {target_columns} not found in data.")
        
        self.data = data.sort_index(ascending=True)
        self.target_columns = target_columns
        self.seq_length = seq_length
        self.scaler = StandardScaler()
        self.model_name = model_name
        self.scaler_path = f'./assets/scaler_{model_name}.pkl'

        # If a scaler file exists, load it. Otherwise, fit a new scaler
        if os.path.exists(self.scaler_path):
            self.load_scaler()
            self.data_scaled = self.scaler.transform(self.data[self.target_columns])
        else:
            self.data_scaled = self.scaler.fit_transform(self.data[self.target_columns])
            self.save_scaler()

    def save_scaler(self):
        logger.info("Saving scaler")
        joblib.dump(self.scaler, self.scaler_path)
        logger.info(f"Scaler saved to {self.scaler_path}")

    def load_scaler(self):
        logger.info("Loading scaler")
        try:
            self.scaler = joblib.load(self.scaler_path)
            logger.info(f"Scaler loaded from {self.scaler_path}")
        except Exception as e:
            logger.error(f"Error loading scaler from {self.scaler_path}: {e}")
            raise

    def split_data(self, test_size=0.2):
        logger.info("Splitting data")
        train_data, test_data = train_test_split(self.data_scaled, test_size=test_size, shuffle=False)
        return train_data, test_data

    def prepare_data_rnn(self, data):
        logger.info("Preparing data for RNN")
        X, y = [], []
        for i in range(self.seq_length, len(data)):
            X.append(data[i-self.seq_length:i])
            y.append(data[i, 0])
        X, y = np.array(X), np.array(y)
        logger.info(f"RNN Data Shape - X: {X.shape}, y: {y.shape}")
        return X, y   

    def prepare_data_for_tree_based_model(self, target_column='Close', test_size=0.2, scale=False):
        logger.info("Preparing data for tree-based model")
        X = self.data.drop(target_column, axis=1)
        y = self.data[target_column]

        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, shuffle=False)

        if scale:
            scaler = StandardScaler()
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)
            
        logger.info(f"Tree Data Shape - X: {X_train.shape}, {y_train.shape} , y: {X_test.shape}, {y_test.shape}")
        return X_train, y_train, X_test, y_test

    def visualize_split(self, train_data, test_data):
        logger.info("Visualizing data split")
        plt.figure(figsize=(14, 7))
        plt.plot(np.arange(len(train_data)), self.scaler.inverse_transform(train_data)[:, 0], color='blue', label='Training Data')
        plt.plot(np.arange(len(train_data), len(train_data) + len(test_data)), self.scaler.inverse_transform(test_data)[:, 0], color='orange', label='Testing Data')
        plt.xlabel('Time Step')
        plt.ylabel('Normalized Value')
        plt.legend()
        plt.show()
    
    def prepare_data_prophet(self, target_col, regressor_cols=None, model_type='uni', normalize=False):
        logging.info("Preparing data for Prophet model")
        if target_col not in self.data.columns:
            logger.error(f"Target column {target_col} not found in data.")
            raise ValueError(f"Target column {target_col} not found in data.")
        if regressor_cols:
            for col in regressor_cols:
                if col not in self.data.columns:
                    logger.error(f"Regressor column {col} not found in data.")
                    raise ValueError(f"Regressor column {col} not found in data.")
        if not isinstance(self.data.index, pd.DatetimeIndex):
            logger.error("Index of DataFrame is not date.")
            raise ValueError("Index of DataFrame is not date.")

        columns_to_keep = ['Date', target_col] + (regressor_cols if regressor_cols else [])
        data_prophet = self.data.reset_index()[columns_to_keep].rename(columns={'Date': 'ds', target_col: 'y'})

        # Normalize if required
        if normalize:
            scaler_path = f'./assets/scaler_{model_type}.pkl'  # update the path here
            if not os.path.exists(scaler_path):
                logger.error(f"No scaler file found at {scaler_path}")
                raise ValueError(f"No scaler file found at {scaler_path}")
            scaler = joblib.load(scaler_path)
            if regressor_cols:
                data_prophet[regressor_cols] = scaler.transform(data_prophet[regressor_cols])
            data_prophet['y'] = scaler.transform(data_prophet[['y']])

        return data_prophet

In [ ]:
crypto = CryptoData("BTC")
data = crypto.get_cryptocmd_data()

In [ ]:
# Instantiate the DataPreprocessing class
target_columns = ['Close']
preprocessor_uni = DataPreprocessing(data, target_columns, model_name='prophet_uni')
data_prophet_uni = preprocessor_uni.prepare_data_prophet(target_col='Close', model_type='uni', normalize=True)
data_prophet_uni

In [ ]:
crypto = CryptoData("BTC")
data = crypto.get_cryptocmd_data()

In [ ]:
target_columns = ['Open', 'High', 'Low', 'Volume', 'Market Cap']  # 'Close' excluded
preprocessor_multi = DataPreprocessing(data, target_columns, model_name='prophet_multi')
preprocessor_multi.save_scaler()  # retrain and save the scaler with 5 columns

regressor_cols = ['Open', 'High', 'Low', 'Volume', 'Market Cap','y']
data_prophet_multi = preprocessor_multi.prepare_data_prophet(target_col='Close', regressor_cols=regressor_cols, model_type='multi', normalize=True)
data_prophet_multi

In [ ]:
crypto = CryptoData("BTC")
data = crypto.get_cryptocmd_data()

In [ ]:
data1 = data.copy()
data1 = data1.drop(['Open', 'High', 'Low', 'Volume', 'Market Cap'], axis=True)
data1.rename(columns = {'Close':'y'}, inplace = True)
data1.index.names = ['ds']
data1 = data1.reset_index()
data1

In [ ]:
import pandas as pd
from prophet import Prophet

# Python
m = Prophet()
m.fit(data1)
# Python
future = m.make_future_dataframe(periods=365)
future.tail()

In [ ]:
# Python
forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [ ]:
fig2 = m.plot_components(forecast)
fig2

In [ ]:
# Python
from prophet.plot import plot_plotly, plot_components_plotly

plot_plotly(m, forecast)
# Python
plot_components_plotly(m, forecast)

In [ ]:
# Python
from prophet.plot import add_changepoints_to_plot
fig = m.plot(forecast)
a = add_changepoints_to_plot(fig.gca(), m, forecast)
fig

In [ ]:
from prophet import Prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import os
import joblib
import logging

class CryptoProphet:
    def __init__(self, model_name, data, target_column, regressor_cols=None):
        logger.info("Initializing Prophet Model")
        self.model_name = model_name
        self.data = data
        self.target_column = target_column
        self.regressor_cols = regressor_cols if regressor_cols else []
        self.model = Prophet()
        self.model_path = f'./trained_models/model_{model_name}.pkl'
        for regressor in self.regressor_cols:
            self.model.add_regressor(regressor)
        
    def train(self):
        logger.info("Training Prophet Model")
        self.model.fit(self.data)

    def save_model(self):
        logger.info("Saving Prophet Model")
        joblib.dump(self.model, self.model_path)
        logger.info(f"Model saved to {self.model_path}")

    def load_model(self):
        logger.info("Loading Prophet Model")
        if os.path.exists(self.model_path):
            self.model = joblib.load(self.model_path)
            logger.info(f"Model loaded from {self.model_path}")
        else:
            logger.error(f"No model file found at {self.model_path}")
            raise ValueError(f"No model file found at {self.model_path}")

    def make_future_dataframe(self, periods, freq='D'):
        future = self.model.make_future_dataframe(periods=periods, freq=freq)
        return future

    def predict(self, future):
        logger.info("Making Predictions")
        forecast = self.model.predict(future)
        return forecast

    def plot_predictions(self, forecast):
        logger.info("Plotting Predictions")
        self.model.plot(forecast)
        plt.show()

    def evaluate(self, forecast):
        logger.info("Evaluating Model")
        y_true = self.data[self.target_column].values
        y_pred = forecast['yhat'].values[:len(y_true)]
        mse = mean_squared_error(y_true, y_pred)
        mae = mean_absolute_error(y_true, y_pred)
        logger.info(f"MSE: {mse}, MAE: {mae}")
        return mse, mae


In [ ]:
# Assuming that data_prophet_uni has been created
prophet_uni = CryptoProphet(model_name='prophet_uni', data=data_prophet_uni, target_column='y')
prophet_uni.train()
prophet_uni.save_model()

# Make future dataframe for predictions
future = prophet_uni.make_future_dataframe(periods=30)

# Predict and plot
forecast = prophet_uni.predict(future)
prophet_uni.plot_predictions(forecast)

# Evaluate model
mse, mae = prophet_uni.evaluate(forecast)


In [ ]:
# Assuming that data_prophet_multi has been created
prophet_multi = CryptoProphet(model_name='prophet_multi', data=data_prophet_multi, target_column='Close', regressor_cols=['Open', 'High', 'Low', 'Volume', 'Market Cap'])
prophet_multi.train()
prophet_multi.save_model()

# Make future dataframe for predictions
future = prophet_multi.make_future_dataframe(periods=30)

# Predict and plot
forecast = prophet_multi.predict(future)
prophet_multi.plot_predictions(forecast)

# Evaluate model
mse, mae = prophet_multi.evaluate(forecast)


In [ ]:
data1 = data.copy()
data1 = data1.drop(['Open', 'High', 'Low', 'Volume', 'Market Cap'], axis=True)
data1.rename(columns = {'Close':'y'}, inplace = True)
data1.index.names = ['ds']
data1 = data1.reset_index()
data1.rename(columns = {'y':'Close','ds':'Date'}, inplace = True)
data1

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Read the data
df = data1.copy()
df["Date"] = pd.to_datetime(df["Date"])
df["Return"] = df["Close"].pct_change()
df["Return"] = df["Return"].fillna(df["Return"].mean())
df = df[np.abs(df["Return"] - df["Return"].mean()) <= 3 * df["Return"].std()]

# Define the number of states for the Markov chain
n_states = 3

# Cluster the returns into n_states using k-means algorithm
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=n_states, random_state=0).fit(df["Return"].values.reshape(-1, 1))
df["State"] = kmeans.predict(df["Return"].values.reshape(-1, 1))

# Plot the states on the price chart
plt.figure(figsize=(12, 8))
plt.plot(df["Date"], df["Close"], label="Price")
for i in range(n_states):
    plt.scatter(df[df["State"] == i]["Date"], df[df["State"] == i]["Close"], label=f"State {i}")
plt.legend()
plt.xlabel("Date")
plt.ylabel("Price")
plt.title("BTC Price and States")
plt.show()

# Estimate the transition matrix for the Markov chain
trans_mat = np.zeros((n_states, n_states))
for i in range(n_states):
    for j in range(n_states):
        # Count the number of transitions from state i to state j
        trans_mat[i, j] = len(df[(df["State"].shift(1) == i) & (df["State"] == j)])
    # Normalize the row to sum to 1
    trans_mat[i, :] = trans_mat[i, :] / trans_mat[i, :].sum()

# Print the transition matrix
print("Transition matrix:")
print(trans_mat)

# Define a function to simulate the Markov chain
def simulate_markov_chain(trans_mat, start_state, n_steps):
    # Initialize an empty list to store the states
    states = []
    # Set the current state to the start state
    current_state = start_state
    # Append the current state to the list
    states.append(current_state)
    # Loop for n_steps times
    for _ in range(n_steps):
        # Draw a random number between 0 and 1
        rand = np.random.rand()
        # Find the next state based on the transition matrix and the random number
        next_state = np.where(trans_mat[current_state, :].cumsum() > rand)[0][0]
        # Set the current state to the next state
        current_state = next_state
        # Append the current state to the list
        states.append(current_state)
    # Return the list of states
    return states

# Simulate the Markov chain for 10 steps starting from state 0
sim_states = simulate_markov_chain(trans_mat, start_state=0, n_steps=10)

# Print the simulated states
print("Simulated states:")
print(sim_states)

# Calculate the expected return for each state based on the historical data
state_returns = df.groupby("State")["Return"].mean()

# Print the expected returns
print("Expected returns:")
print(state_returns)

# Calculate the predicted price for each simulated state based on the last price and the expected return
last_price = df["Close"].iloc[-1]
pred_prices = [last_price]
for state in sim_states:
    pred_price = pred_prices[-1] * (1 + state_returns[state])
    pred_prices.append(pred_price)

# Print the predicted prices
print("Predicted prices:")
print(pred_prices)

# Plot the predicted prices on a chart
plt.figure(figsize=(12, 8))
plt.plot(range(len(pred_prices)), pred_prices, marker="o", label="Predicted")
plt.xlabel("Step")
plt.ylabel("Price")
plt.title("BTC Price Prediction using Markov Chain")
plt.show()


In [ ]:
data1

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Read the data
df = data1.copy()
# Convert the date column to datetime format
df["Date"] = pd.to_datetime(df["Date"])

# Define the smoothing parameter alpha
alpha = 0.2

# Initialize the first smoothed value as the first price
smoothed = [df["Close"].iloc[0]]

# Loop through the rest of the prices
for i in range(1, len(df)):
    # Apply the exponential smoothing formula
    smoothed_value = alpha * df["Close"].iloc[i] + (1 - alpha) * smoothed[-1]
    # Append the smoothed value to the list
    smoothed.append(smoothed_value)

# Add the smoothed values as a new column to the dataframe
df["Smoothed"] = smoothed

# Plot the price and the smoothed values on a chart
plt.figure(figsize=(12, 8))
plt.plot(df["Date"], df["Close"], label="Price")
plt.plot(df["Date"], df["Smoothed"], label="Smoothed")
plt.legend()
plt.xlabel("Date")
plt.ylabel("Price")
plt.title("BTC Price and Exponential Smoothing")
plt.show()

# Predict the next price using the last smoothed value
pred_price = df["Smoothed"].iloc[-1]

# Print the predicted price
print("Predicted price:")
print(pred_price)


In [ ]:
data1 = data.copy()
data1 = data1.drop(['Open', 'High', 'Low', 'Volume', 'Market Cap'], axis=True)
data1.rename(columns = {'Close':'y'}, inplace = True)
data1.index.names = ['ds']
data1 = data1.reset_index()
data1.rename(columns = {'y':'Close','ds':'Date'}, inplace = True)
data1

In [ ]:
# Import packages and dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense

# Load and preprocess the data
data1["Date"] = pd.to_datetime(data1["Date"]) # Convert the Date column to datetime format
data1 = data1.set_index("Date") # Set the Date column as the index
data1 = data1.sort_index() # Sort the data by date
data1 = data1[["Close"]] # Select only the Close column

# Split the data into train and test sets
train_size = int(len(data1) * 0.8) # Use 80% of the data for training
train, test = data1[0:train_size], data1[train_size:len(data1)] # Split the data
print("Train size:", len(train))
print("Test size:", len(test))

# Scale the data to the range [0, 1]
scaler = MinMaxScaler(feature_range=(0, 1)) # Create a scaler object
train_scaled = scaler.fit_transform(train) # Fit and transform the train set
test_scaled = scaler.transform(test) # Transform the test set

# Define a function to create input-output pairs from a time series
def create_dataset(dataset, look_back=1):
    X, Y = [], [] # Initialize empty lists for input and output
    for i in range(len(dataset) - look_back - 1): # Loop over the dataset
        X.append(dataset[i:(i + look_back), 0]) # Append the previous values as input
        Y.append(dataset[i + look_back, 0]) # Append the current value as output
    return np.array(X), np.array(Y) # Return numpy arrays

# Create input-output pairs for train and test sets
look_back = 3 # Use 3 previous values to predict the next one
X_train, y_train = create_dataset(train_scaled, look_back) # Create train set
X_test, y_test = create_dataset(test_scaled, look_back) # Create test set

# Reshape the input to fit the MLP model
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1) # Add a dimension for features
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1) # Add a dimension for features

# Define and compile the MLP model
model = Sequential() # Create a sequential model
model.add(Dense(10, input_shape=(look_back, 1), activation="relu")) # Add a dense layer with 10 neurons and relu activation
model.add(Dense(5, activation="relu")) # Add another dense layer with 5 neurons and relu activation
model.add(Dense(1)) # Add an output layer with 1 neuron and no activation
model.compile(loss="mean_squared_error", optimizer="adam") # Compile the model with MSE loss and adam optimizer

# Fit the model on the train set
model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=2) # Train the model for 50 epochs with batch size of 32

# Evaluate the model on the test set
y_pred = model.predict(X_test) # Predict the test set
y_pred = y_pred.reshape(y_pred.shape[0], y_pred.shape[1]) # Reshape y_pred to 2D
y_pred = scaler.inverse_transform(y_pred) # Inverse transform the predictions to original scale
y_test = scaler.inverse_transform([y_test]) # Inverse transform the test set to original scale
test_score = mean_squared_error(y_test[0], y_pred[:,0]) # Calculate the MSE score
print("Test MSE:", test_score)

# Plot the actual and predicted values
plt.plot(data1.index[train_size + look_back + 1:], y_test[0], label="Actual") # Plot the actual values
plt.plot(data1.index[train_size + look_back + 1:], y_pred[:,0], label="Predicted") # Plot the predicted values
plt.xlabel("Date") # Set x-axis label
plt.ylabel("Close") # Set y-axis label
plt.legend() # Show legend
plt.show() # Show plot

In [3]:
%conda install pytorch

In [11]:
# Import packages
import torch
import pytorch_forecasting
from pytorch_forecasting import TemporalFusionTransformer

ModuleNotFoundError: No module named 'pytorch_forecasting'

In [ ]:
# Import packages
import torch
import pytorch_forecasting
from pytorch_forecasting import TemporalFusionTransformer

# Define parameters
max_encoder_length = 60 # maximum length of encoder sequence
max_prediction_length = 20 # maximum length of prediction sequence
hidden_size = 16 # hidden size of TFT model
dropout = 0.1 # dropout rate of TFT model
learning_rate = 0.01 # learning rate for optimizer
weight_decay = 1e-5 # weight decay for optimizer
max_epochs = 10 # maximum number of epochs for training

# Define time index column and target column
time_idx = "Date"
target = "Close"

# Define categorical and continuous variables
categorical_variables = [] # list of categorical column names, if any
continuous_variables = [] # list of continuous column names, if any

# Define group ids for each series, if any
group_ids = [] # list of group id column names, if any

# Create TimeSeriesDataSet from data1 dataframe
dataset = pytorch_forecasting.TimeSeriesDataSet(
    data=data1,
    time_idx=time_idx,
    target=target,
    group_ids=group_ids,
    max_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    categorical_encoders={var: pytorch_forecasting.encoders.NaNLabelEncoder(add_nan=True) for var in categorical_variables},
    time_varying_known_categoricals=categorical_variables,
    time_varying_known_reals=continuous_variables,
    time_varying_unknown_reals=[target],
    target_normalizer=pytorch_forecasting.data.encoders.TorchNormalizer(),
)

# Create dataloaders for training and validation sets
train_dataloader = dataset.to_dataloader(train=True, batch_size=32, num_workers=0)
val_dataloader = dataset.to_dataloader(train=False, batch_size=32 * 10, num_workers=0)

# Create TFT model
model = TemporalFusionTransformer.from_dataset(
    dataset,
    hidden_size=hidden_size,
    dropout=dropout,
    output_size=dataset.target_normalizer.mean.size(0),
    loss=pytorch_forecasting.metrics.RMSE(),
    log_interval=10,
    reduce_on_plateau_patience=4,
)

# Create optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

# Train the model
model.fit(train_dataloader, val_dataloader, max_epochs=max_epochs)

# Make predictions on new data
predictions = model.predict(val_dataloader)


ImportError: cannot import name 'pytorch_forecasting' from 'torch' (/opt/conda/lib/python3.7/site-packages/torch/__init__.py)

# Block 8 - FB Prophet Models

In [ ]:
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import numpy as np


class ProphetModel:
    def __init__(self, df):
        df.reset_index(level=0, inplace=True)
        self.df = df.rename(columns={"Date": "ds", "Close": "y"})
        self.model = Prophet()

    def train_model(self):
        self.model.fit(self.df)

    def make_forecast(self, periods):
        future = self.model.make_future_dataframe(periods=periods)
        forecast = self.model.predict(future)
        self.forecast = forecast
        return forecast  # Return the forecast DataFrame


    def plot_forecast(self):
        return self.model.plot(self.forecast)

    def evaluate_model(self):
        df_cv = cross_validation(self.model, horizon='30 days')
        df_p = performance_metrics(df_cv)
        
        rmse = np.mean(df_p['rmse'])
        mae = np.mean(df_p['mae'])
        mse = np.mean(df_p['mse'])
        r2 = np.mean(df_p['coverage'])

        eval_df = pd.DataFrame({
            'Metric': ['RMSE', 'MAE', 'MSE', 'R2'],
            'Value': [rmse, mae, mse, r2]
        })

        return eval_df

    def plot_residuals(self):
        df_res = self.df.copy()
        df_res['yhat'] = self.forecast['yhat']
        df_res['residuals'] = df_res['y'] - df_res['yhat']
        plt.figure(figsize=(10,6))
        plt.plot(df_res['ds'], df_res['residuals'], label='Residuals')
        plt.title('Residuals over time')
        plt.show()
    
    def get_predictions_df(self, periods):
        df_pred = self.make_forecast(periods)
        compare_df = df_pred.set_index('ds')[['yhat']].join(self.df.set_index('ds'))
        compare_df = compare_df.rename(columns={'yhat': 'Predicted', 'y': 'Actual'})
        return compare_df


In [ ]:
# Create an instance of the ProphetModel class
univariate_model = ProphetModel(data)
# Train the model
univariate_model.train_model()
# Make a forecast
univariate_model.make_forecast(30) # adjust the number of periods as needed
# Evaluate the model
evaluation = univariate_model.evaluate_model()
print(evaluation)
# Plot residuals
univariate_model.plot_residuals()
# Get the predictions DataFrame
predictions_df = univariate_model.get_predictions_df(30) # adjust the number of periods as needed
print(predictions_df)

# Block 9 - AMZ Deep Time Models

In [ ]:
import pandas as pd
from gluonts.dataset.common import ListDataset
from gluonts.model.deepar import DeepAREstimator
from gluonts.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions

class DeepTimeModel:
    def __init__(self, df, column_name):
        self.df = df
        self.column_name = column_name
        self.model = None
        self.predictor = None
        self.forecast = None

    def train_model(self, prediction_length):
        dataset = ListDataset([{
            "start": self.df.index[0],
            "target": self.df[self.column_name]
        }], freq = "D")
        
        estimator = DeepAREstimator(freq="D", 
                                    prediction_length=prediction_length, 
                                    trainer=Trainer(epochs=10))
        self.model = estimator.train(training_data=dataset)
    
    def get_predictions(self, test_data):
        predictor = self.model.predict(test_data)
        self.predictor = predictor
        return predictor


In [ ]:
# Create an instance of the DeepTimeModel class
deeptime_model = DeepTimeModel(df, 'column_name')
# Train the model
deeptime_model.train_model(prediction_length=30)
# Get the predictions
predictions = deeptime_model.get_predictions(test_data)
